In [2]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, set_seed
import soundfile as sf

from IPython.display import Audio


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Flash attention 2 is not installed


In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# prompt = "我是中天新闻主播，为您带来今天的新闻。"
prompt = "wo3 shi4 zhong1 tian1 xin1 wen2 zhu3 bo1 ， wei2 nin2 dai4 lai2 jin1 tian1 de xin1 wen2"
# prompt = "zui4 xin1 ou1 zhou1 mo2 shi4 mo2 ni3 xian3 shi4 ， jin1 ri4 bei3 bu4 、 dong1 ban4 bu4 ou3 you3 ju2 bu4 duan3 zan4 jiang4 yu3 ； zhong1 nan2 bu4 duo1 yun2 shi2 qing2 、 wu3 hou4 jin4 shan1 qu1 you3 ju2 bu4 zhen4 yu3 huo4 lei2 yu3 ， zhu4 yi4 da4 lei2 yu3 。"
# prompt = "ka2 er2 pu3 pei2 wai4 sun1 wan2 hua2 ti1"
description = "Ling speaks at an average pace with an animated delivery in a very confined sounding environment with clear audio quality"
tokenizer = AutoTokenizer.from_pretrained("/mydev/parler-tts/output_ling")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
tokenizer.decode(tokenizer(prompt).input_ids)



'wo3 shi4 zhong1 tian1 xin1 wen2 zhu3 bo1, wei2 nin2 dai4 lai2 jin1 tian1 de xin1 wen2 <unk></s>'

In [21]:

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model = ParlerTTSForConditionalGeneration.from_pretrained("/mydev/dataspeech/weights/parler-tts-mini-jenny-30H").to(device)
# tokenizer = AutoTokenizer.from_pretrained("/mydev/dataspeech/weights/parler-tts-mini-jenny-30H")
model = ParlerTTSForConditionalGeneration.from_pretrained("/mydev/parler-tts/output_ling").to(device)
tokenizer = AutoTokenizer.from_pretrained("/mydev/parler-tts/output_ling")

# prompt = "卡尔普"
# prompt = "ka2 er2 pu3 pei2 wai4 sun1 wan2 hua2 ti1"
# prompt = "wo3 shi4 zhong1 tian1 xin1 wen2 zhu3 bo1 ， wei2 nin2 dai4 lai2 jin1 tian1 de xin1 wen2"
prompt = "zuo2 tian1 wan3 shang4 xia4 liao3 yi1 chang2 da4 yu3 ， zao3 shang4 tian1 kong1 hen3 qing2 lang3"
# description = "Jenny speaks at an average pace with an animated delivery in a very confined sounding environment with clear audio quality"
# prompt = "我是中天新闻主播，为您带来今天的新闻。"
description = "Ling speaks at an slow pace with an animated delivery in a very confined sounding environment with clear audio quality"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

set_seed(42)
# specify min length to avoid 0-length generations
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids, min_length=10)
audio_arr = generation.cpu().numpy().squeeze()
# sf.write("/mydev/dataspeech/chinese.wav", audio_arr, model.config.sampling_rate)


Both `min_new_tokens` (=10) and `min_length`(=10) seem to have been set. `min_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [5]:
input_ids.shape

torch.Size([1, 25])

In [6]:
prompt_input_ids.shape

torch.Size([1, 55])

In [22]:
Audio(audio_arr, rate=model.config.sampling_rate)

In [23]:
sf.write("/mydev/dataspeech/chinese.wav", audio_arr, model.config.sampling_rate)
